<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [298]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko

In [299]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [300]:
#@title Available Coins { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#General Information table

def coins_stats(url):
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    
    token_list = pd.DataFrame(data)

    print('Stats: ')
    print(token_list.info(verbose=True))
    print(token_list.isnull().sum())
    print(token_list.describe())
   
    return token_list 


def main():
  url = "https://api.coingecko.com/api/v3/coins/list"

  coins_stats(url)
  

if __name__ == "__main__":
    main()

Stats: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13491 entries, 0 to 13490
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13491 non-null  object
 1   symbol  13491 non-null  object
 2   name    13491 non-null  object
dtypes: object(3)
memory usage: 316.3+ KB
None
id        0
symbol    0
name      0
dtype: int64
            id symbol    name
count    13491  13491   13491
unique   13491  11217   13491
top     01coin   moon  01coin
freq         1     10       1


In [336]:
#@title  Download Coins Data Reports { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

 #suppressing carriers
id = token_list.id[:5]
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list
try:

  start = 0
  ends =   2000

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}/market_chart?vs_currency={}&days={}".format(i,currency,Horizon))
    output = response.text
    json_parsed = json.loads(output)
    print(json_parsed)

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    full_table['id'] = i
    #full_table.loc(i)
    frames.append(full_table)

    #Slow down requestsloc

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
df = single

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_excel(path + 'CoinGecko_prices_{}-{}.xlsx'.format(start,ends))

len(single)

{'prices': [[1536969600000, 0.009788403063705275], [1537056000000, 0.009882558743828542], [1537142400000, 0.012145158274011318], [1537228800000, 0.007965204513941592], [1537315200000, 0.011866053006407695], [1537401600000, 0.011457442133231787], [1537488000000, 0.009856303171318184], [1537574400000, 0.010346361715446901], [1537660800000, 0.012509513238031308], [1537747200000, 0.00872956465326417], [1537833600000, 0.01005844860946907], [1537920000000, 0.008786264540711866], [1538006400000, 0.008104729360386247], [1538092800000, 0.00929708644129806], [1538179200000, 0.008332842978249768], [1538265600000, 0.00710890526630388], [1538352000000, 0.00704758470712929], [1538438400000, 0.010964302136757384], [1538524800000, 0.00783325848098873], [1538611200000, 0.00727669373334877], [1538697600000, 0.007948086657450218], [1538784000000, 0.010100972410891288], [1538870400000, 0.012831612472876296], [1538956800000, 0.011294693135382498], [1539043200000, 0.013286846597662993], [1539129600000, 0.01

5

In [339]:
import numpy as np
import pandas as pd

frames=[]

for rows in single['id']:
  for i in single['prices']:
      
    df = pd.DataFrame(i)
    df['id'] = rows

  frames.append(df)

combined = pd.concat(frames)

tokens = len(combined['id'].unique())

combined.to_csv(path+'/CoinGecko_prices.csv')
combined.info()

print('REPORT ')
print('Tokens total: '+ str(tokens))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190 entries, 0 to 237
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1190 non-null   int64  
 1   1       1190 non-null   float64
 2   id      1190 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 37.2+ KB
REPORT 
Tokens total: 5


In [341]:
combined


,0,1,id
0,1596672000000,7240.231698,01coin
1,1596758400000,7240.231698,01coin
2,1596844800000,6976.395764,01coin
3,1596931200000,7106.512949,01coin
4,1597017600000,7106.512949,01coin
...,...,...,...
233,1648252800000,8780.347729,0-5x-long-bitcoin-cash-token
234,1648339200000,8780.347729,0-5x-long-bitcoin-cash-token
235,1648425600000,9070.274067,0-5x-long-bitcoin-cash-token
236,1648503744000,9070.274067,0-5x-long-bitcoin-cash-token


In [ ]:
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_excel(path + 'CoinGecko_prices_{}-{}.xlsx'.format(start,ends))